# MHPI hydroDL2.0 Tutorial: **dHBV1.1p**
---

This is a basic implementation of the generic differentiable modeling framework `dMG` using the HBV1.1p hydrology model
plugin from the `hydroDL2.0` repository.



Last Revision: 30 Oct. 2024

Authors; Leo Lonzarich

---

## 1. Basic Hands-off Deployment:

In this first demonstration, we show how `dMG` using a HBV1.1p physics model backbone from `hydroDL2` can be operated
in a  few steps. These are outlined as follows:

0. First, ensure that you have the correct *env* configured. To avoid manually downloading required Python packages,
create a `hydrodl` env using 

    `conda env create -f envs/hydrodl_env.yaml`.

    Once activated, confirm PyTorch installed correctly with `torch.cuda.is_available()`. If this reports false, try
    - `conda uninstall pytorch`
    - `conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia`

1. Set your desired model and experiment configuration settings within a *yaml* config file.
    - For this tutorial, you can find this config located at `generic_diffModel/example/conf/dhbv_11p_config.yaml`. Note,
    this yaml is configured to reproduce dHBV1.1p benchmarks for 531 CAMELS basins, trained and tested for 9 and 10 years,
    respectively
    - For normal operation of `dMG`, however, see `generic_diffModel/conf/config.yaml`.
2. Either run `python dMG/__main__.py` in your terminal, or (recommended) run the contents of `__main__.py` in the cells below.
    - This will parse your config into a dictionary, load the HBV1.1p hydrology model, and begin training or testing.


### 1.1 Create Configurations Dictionary

The first cell below will convert the configurations yaml file into a key-indexed dictionary, with keys being the
config settings. 

That is, if `mode: train` is set in `dhbv_11p_config.yaml`, the dictionary will yield `config['mode'] == 'train'`.
Similarly, `training: start_time: 1999/10/01` is equivalent to `config['training']['start_time'] == '1999/10/01'`.


In [1]:
## Load in the dMG configuration file with dHBV1.1p options:
import hydra
from omegaconf import DictConfig, OmegaConf



# Example configs stored in /example/conf
CONFIG_PATH = '../conf'
CONFIG_NAME = 'dhbv_11p_config'



def load_config(config_path: str, config_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=config_path, version_base='1.3'):
        config = hydra.compose(config_name=config_name)
   
    config_dict = OmegaConf.to_container(config, resolve=True)
    return config_dict

config = load_config(CONFIG_PATH, CONFIG_NAME)


### 1.2 Run `__main__.py` with Configurations

This code instantiates a model Trainer which will train or test a model per the user's specification in the config. Note
that `__main__.py` is trimmed-down here to illustrate it's primary objective.
Within the Trainer itself, 
- CAMELS data will be loaded and preprocessed,
- A differenial model object with the HBV1.1p backbone will be created, and 
- An optimizer and loss function will be initialized.

These and other details/structure of `dMG` will be illustrated in the second part of this tutorial.


In [2]:
import sys
sys.path.append('../../dMG') # Add the root directory of dMG to the path

import torch
import logging
from typing import Any, Dict
from conf.config import ModeEnum
from trainers import build_handler
from core.utils import (create_output_dirs, set_randomseed, set_system_spec,
                        print_config)

log = logging.getLogger(__name__)



def run_train_test(config_dict: Dict[str, Any]) -> None:
    """
    Run training and testing as one experiment.
    """
    # Training
    config_dict['mode'] = ModeEnum.train
    train_experiment_handler = build_handler(config_dict)
    train_experiment_handler.run()

    # Testing
    config_dict['mode'] = ModeEnum.test
    test_experiment_handler = build_handler(config_dict)            
    test_experiment_handler.dplh_model_handler = train_experiment_handler.dplh_model_handler
    test_experiment_handler.run()


def run_experiment(config_dict: Dict[str, Any]) -> None:
    """ Run an experiment based on the mode specified in the configuration. """
    experiment_handler = build_handler(config_dict)
    experiment_handler.run()



# Set device, dtype, output directories, and random seed.
set_randomseed(config['random_seed'])

config['device'], config['dtype'] = set_system_spec(config['gpu_id'])
config = create_output_dirs(config)

log.info(f"RUNNING MODE: {config['mode']}")
print_config(config)

# Run training and testing together, or one at a time.
if config['mode'] == ModeEnum.train_test:
    run_train_test(config)

else:
    run_experiment(config)

torch.cuda.empty_cache()


Current Configuration
  Experiment Mode:    train               
  Ensemble Mode:      none                
  Model 1:            HBV                 

Data Loader
  Data Source:        camels_531          
  Train Range :       1999/10/01          2008/10/01          

Model Parameters
  Train Epochs:       50                  Batch Size:         100                 
  Dropout:            0.5                 Hidden Size:        256                 
  Warmup:             365                 Concurrent Models:  16                  
  Optimizer:          RmseLossFlowComb    

Machine
  Use Device:         cuda                



Epoch 1/50:   0%|          | 0/194 [00:00<?, ?it/s]/data/lgl5139/project_blue_eyes/generic_diffModel/example/dPLHBV/../../dMG/models/neural_networks/lstm_models.py:103: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /opt/conda/conda-bld/pytorch_1729647348947/work/aten/src/ATen/native/cudnn/RNN.cpp:1410.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
Epoch 1/50:   1%|          | 1/194 [00:03<12:01,  3.74s/it]

Batch loss:  2.5342836380004883


Epoch 1/50:   1%|          | 2/194 [00:05<07:36,  2.38s/it]

Batch loss:  2.2687909603118896


...

Epoch 1/50:   7%|▋         | 14/194 [00:24<05:01,  1.68s/it]

Batch loss:  1.8150919675827026


Epoch 1/50:   8%|▊         | 15/194 [00:25<04:54,  1.65s/it]

Batch loss:  1.9237264394760132


KeyboardInterrupt: 

### 1.3 Get Results of Tested Model

If you have run testing on a trained model and want to view the results, you can find a `mstd.csv` file in your model
directory, which will give you the statistics on your model's performance. 

*Graphical visualizations of model output will be supported in a future updated.*


---

## 2. Breakdown of Intermediate Steps: Training

In this example, we break down dHBV1.1p differentiable model training in `dMG` by exposing the internals of the Trainer.
(**Note**, we are bypassing `__main__.py` in this part since it simply runs the Trainer.)

### 2.1 Create Configurations Dictionary

Once again, begin by creating a configurations dictionary.

In [20]:
## Load in the dMG configuration file with dHBV1.1p options:
from omegaconf import DictConfig, OmegaConf

import hydra


# Example configs stored in /example/conf
CONFIG_PATH = '../conf'
CONFIG_NAME = 'dhbv_11p_config'



def load_config(config_path: str, config_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=config_path, version_base='1.3'):
        config = hydra.compose(config_name=config_name)
   
    config_dict = OmegaConf.to_container(config, resolve=True)
    return config_dict

config = load_config(CONFIG_PATH, CONFIG_NAME)


### 2.2 Initialize model, optimizer and loss function

These are the auxillary tasks first completed by the Trainer.